In [1]:
#Import necessary libraries
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from kneed import KneeLocator
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from yellowbrick.cluster import KElbowVisualizer
import plotly.graph_objs as go
from plotly import tools
from plotly.subplots import make_subplots
import plotly.offline as py
import plotly.express as px
import plotly.io as pio
pio.templates

Templates configuration
-----------------------
    Default template: 'plotly'
    Available templates:
        ['ggplot2', 'seaborn', 'simple_white', 'plotly',
         'plotly_white', 'plotly_dark', 'presentation', 'xgridoff',
         'ygridoff', 'gridon', 'none']

In [2]:
# Create lists with machines and machine types:
machine_files = ['./df_6dB_valve_fe_ta_imb.csv', './df_6dB_pump_fe_ta_imb.csv', './df_6dB_slider_fe_ta_imb.csv', './df_6dB_fan_fe_ta_imb.csv']
machine_types = ['id_00', 'id_02', 'id_04', 'id_06']

In [5]:
def get_corr(df, file_path, machine_type):
    df_machine_type = df[df.machine_type == machine_type]
    df_machine_type.reset_index(inplace = True)
    df_machine_type.drop(['machine_type'],axis = 1, inplace = True) 
    df_machine_type_features = df_machine_type.iloc[:,:-1]
    # z-score normalisation per machine
    metrics_normalized_machine_type =(df_machine_type_features-df_machine_type_features.mean())/df_machine_type_features.std()
    metrics_normalized_machine_type = metrics_normalized_machine_type.rename(columns={'mfccs': 'mfccs_zscore','zero_cross': 'zero_cross_zscore','rms': 'rms_zscore'})
    metrics_normalized_machine_type.reset_index(drop=True, inplace = True)
    # fit a kmeans object to the dataset
    kmeans = KMeans(n_clusters=2, init='k-means++').fit(metrics_normalized_machine_type)

    # clusters is an attribute of the object
    cluster_centers = kmeans.cluster_centers_
    # add cluster index to dataframe

    cluster_labels = pd.Series(kmeans.labels_, name='cluster')
    metrics_normalized_machine_type_kmeans = metrics_normalized_machine_type.join(cluster_labels.to_frame())
    metrics_normalized_machine_type_kmeans['target'] = df_machine_type.target
    correlation = abs(metrics_normalized_machine_type_kmeans.target.corr(metrics_normalized_machine_type_kmeans.cluster))
    return {
        'file_path' : file_path,
        'machine_type': machine_type,
        'correlation': correlation
    }
df = pd.read_csv(machine_files[0])
df.drop(['Unnamed: 0'],axis = 1, inplace = True)
get_corr(df, machine_files[0], machine_types[0])

c:\users\kasia\pyproj\machine-monitoring-conditions-unsupervised\my_env\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



{'file_path': './df_6dB_valve_fe_ta_imb.csv',
 'machine_type': 'id_00',
 'correlation': 0.5420905373277507}

In [6]:
corr = []
for file_path in machine_files: 
    #Read and clean data
    df = pd.read_csv(file_path)
    df.drop(['Unnamed: 0'],axis = 1, inplace = True)
    #df = df_original.drop(['target'],axis = 1)
    for machine_type in machine_types:  
        corr.append(get_corr(df, file_path, machine_type))

c:\users\kasia\pyproj\machine-monitoring-conditions-unsupervised\my_env\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\users\kasia\pyproj\machine-monitoring-conditions-unsupervised\my_env\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\users\kasia\pyproj\machine-monitoring-conditions-unsupervised\my_env\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

In [7]:
df_corr = pd.DataFrame(corr)

In [8]:
df_corr.head(16)

,file_path,machine_type,correlation
0,./df_6dB_valve_fe_ta_imb.csv,id_00,0.542091
1,./df_6dB_valve_fe_ta_imb.csv,id_02,0.119709
2,./df_6dB_valve_fe_ta_imb.csv,id_04,0.427272
3,./df_6dB_valve_fe_ta_imb.csv,id_06,0.214341
4,./df_6dB_pump_fe_ta_imb.csv,id_00,0.796292
5,./df_6dB_pump_fe_ta_imb.csv,id_02,0.587637
6,./df_6dB_pump_fe_ta_imb.csv,id_04,1.000000
7,./df_6dB_pump_fe_ta_imb.csv,id_06,0.533319
8,./df_6dB_slider_fe_ta_imb.csv,id_00,1.000000
9,./df_6dB_slider_fe_ta_imb.csv,id_02,0.784465
